In [ ]:
import os
import io
import sys
import json
import codecs
import pandas as pd
from pathlib import Path
from IPython.display import display
from IPython.display import HTML
from IPython.display import clear_output
from traitlets import traitlets

# new deps
# ipywidgets
# jupyter labextension install @jupyter-widgets/jupyterlab-manager


sys.path.append(os.path.join(os.getcwd(),'speciesprimer', 'pipeline'))
# for connection using SSL certificates
%env SSL_CERT_DIR=/etc/ssl/certs
%env REQUESTS_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt

import speciesprimer
# reload for changes in code
import importlib
importlib.reload(speciesprimer)

from speciesprimer import dict_path
from basicfunctions import GeneralFunctions as G
from basicfunctions import HelperFunctions as H

from ipywidgets import widgets, interactive, interactive_output, Layout, VBox, HBox, Label


layout = Layout(width="auto")

In [ ]:
import newgui_classes
importlib.reload(newgui_classes)

from newgui_classes import SpeciesPrimerConfiguration
from newgui_classes import WidgetDesigner
from newgui_classes import SettingsStorage
from newgui_classes import CheckConfigFiles
from newgui_classes import Settings
from newgui_classes import TargetSelection
settings_order, raw_widgets, widget_list, titles = WidgetDesigner().organize_widgets()
conf_settings = SettingsStorage(settings_order, raw_widgets, widget_list, titles)
targetselection = TargetSelection(conf_settings)


/home/ags-bioinfo/blastdb/primerdesign

Lactobacillus helveticus

In [ ]:
class StartPipelineRuns(object):
    def __init__(self, configstore):
        self.configstore = configstore
        self.speciesnames = widgets.SelectMultiple(
            options=self.configstore.specieslist,
            value=self.configstore.specieslist,
            description='Species:',
            disabled=False
            )
        self.start_btn = widgets.Button()
        display(self.speciesnames)
        
        
StartPipelineRuns(conf_settings);

In [ ]:
display(HTML("<h2>SpeciesPrimer configuration: </h2>"))
SpeciesPrimerConfiguration().pipeline_configuration()
display(HTML("<h2>Select targets and settings: </h2>"))
targetselection.new_targets()
targetselection.old_targets()

In [ ]:
conf_settings.settings_dict["Lactobacillus_delbrueckii"]["customdb"]
config.__dict__

In [ ]:
from basicfunctions import HelperFunctions as H

import scripts.configuration
importlib.reload(scripts.configuration)

from scripts.configuration import Config
from scripts.configuration import RunConfig
from scripts.configuration import CLIconf

import scripts.datacollection
importlib.reload(scripts.datacollection)

import scripts.qualitycontrol
importlib.reload(scripts.qualitycontrol)

import scripts.coregenes
importlib.reload(scripts.coregenes)


from scripts.datacollection import GenomeDownload
from scripts.datacollection import Annotation
from scripts.qualitycontrol import QualityControl
from scripts.coregenes import PangenomeAnalysis


class RunViz(object):
    def __init__(self):
        self.wids = []
        self.dicts = []
        self.dash = widgets.Tab()
        self.species = []
      
    def create_viz(self, species, configs):
        self.species = species
        for i, sp in enumerate(species):
            d = {}
            stages = ["DataCollection", "Annotation", "QualityControl", "PangenomeAnalysis"]
            suboutput = []
            subprogress = []
            d.update({"DataCollection": GenomeDownload(configs[i])})
            d.update({"Annotation": Annotation(configs[i])})
            d.update({"QualityControl": QualityControl(configs[i])})
            d.update({"PangenomeAnalysis": PangenomeAnalysis(configs[i])})
            self.dicts.append(d)
            for stage in stages:
                suboutput.append(d[stage].output)
                subprogress.append(d[stage].progress)
            
        
            subdashboard = []
            for n, out in enumerate(suboutput):
                accordion = widgets.Accordion(
                    children=[out], selected_index=None)
                accordion.set_title(0, stages[n])
                subprog_dash = HBox([subprogress[n], Label(stages[n] + " progress")])
                subdashboard.append(
                    VBox([accordion, subprog_dash]))
                  
            self.wids.append(VBox(subdashboard))
        self.dash.children = self.wids
        for i, sp in enumerate(species):
            self.dash.set_title(i, sp)

class OverallProgress(object):
    def __init__(self, RunViz):
        self.RV = RunViz
        self.statusprogress = widgets.FloatProgress(value=0, min=0.0, max=1.0)

        self.speciesprogress = HBox([
            VBox([widgets.FloatProgress(value=0, min=0.0, max=1.0) for d in self.RV.dicts]),
            VBox([Label(sp) for sp in self.RV.species])])
        display(HBox([self.statusprogress, Label("Progress")]))
        display(self.speciesprogress)
        
    def update_progress(self):
        vals = [d[k].progress.value for d in RV.dicts for k in d.keys()]
        total = len(vals)
        update = sum(vals)
        self.statusprogress.value = float(update/total)
        for i, d in enumerate(self.RV.dicts):
            vals = [d[k].progress.value for k in d.keys()]
            total = len(vals)
            update = sum(vals)
            self.speciesprogress.children[0].children[i].value = float(update/total)
            
outputlayout={'border': '1px solid black', 'width': '100%', 'height': '160px','overflow': 'auto'}
configs = []
species = ["Lactobacillus_delbrueckii", "Lactobacillus_helveticus"]
for target in species:
    conf = Config(mode="auto", config_dict=conf_settings.settings_dict)
    configuration = conf.get_config(target) 
    nontargetlist = H.create_non_target_list(target)
    config = CLIconf(*configuration, nontargetlist)
    config.set_gui()
    configs.append(config)
    
RV = RunViz()
RV.create_viz(species=species, configs=configs)

OP = OverallProgress(RV)
display(RV.dash)


In [ ]:
stages = ["DataCollection", "Annotation", "QualityControl", "PangenomeAnalysis"]
for i, sp in enumerate(species):
    print(sp)
    for stage in stages:
        print(stage)
        RV.dicts[i][stage].main()
        OP.update_progress()


In [ ]:
   print(datacollecter)

    self.species = species
    outputlayout={'border': '1px solid black', 'width': '100%', 'height': '160px','overflow': 'auto'}
    for sp in species:
        d = {
            "DataCollection": ""}


        subwids = []
        for k in d.keys():
            output = widgets.Output(layout=outputlayout)
            d.update({k: ProgressTest(output)})
            subwids.append(HBox([output, d[k].progress]))
        self.dicts.append(d)
        self.wids.append(VBox(subwids))
    self.dash.children = self.wids
    for i, sp in enumerate(species):
        self.dash.set_title(i, sp)

In [ ]:
display(HTML("<h2>Start SpeciesPrimer runs: </h2>"))
#if targetspecies.settings_dict != {}:
#    targets = list(targetspecies.settings_dict.keys())
#    display(HTML("<h4>" + ", ".join(targets) + "</h4>"))

In [ ]:
"/home/ags-bioinfo/blastdb/primerdesign"

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    total = 100
    for i in range(total):
        time.sleep(0.2)
        progress.value = float(i+1)/total

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

In [ ]:
#dictionaries should persist in primerdesign directory (read/write conflicts with Singularity and Stuff)

In [ ]:
import asyncio
help(asyncio)

In [ ]:
%gui asyncio
from ipywidgets import Button
import asyncio

output = widgets.Output()
pausebutton = Button(description="Pause")
continuebutton = Button(description="Continue")

def wait_for_change(button):
    future = asyncio.Future()
    def getvalue(event):
        future.set_result("Continue")
        button.on_click(getvalue, remove=True) 
    button.on_click(getvalue)
    return future

def continue_button():
    return

async def f():
    print("Paused")
    print()
    x = await wait_for_change(continuebutton)
    print(x)
    print()

def hit_pause(event):
    asyncio.create_task(f())

def some_function():
    for i in range(100):
        print(i)
        time.sleep(0.5)
    
def some_process():

    display(HBox([pausebutton, continuebutton]))
    display(output)
    
    pausebutton.on_click(hit_pause)
    some_function()
    
    
    
    
some_process()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
output = widgets.Output()

@output.capture(clear_output=False, wait=True) # based on https://github.com/jupyter-widgets/ipywidgets/issues/1846 and https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
def sayHello(b):
    print("You have pushed the button, congratulations I guess.")

run_button = widgets.Button(
      description = 'continue'
)
print("Hello, please push the continue button")
run_button.on_click(sayHello)
display(run_button)
output

In [ ]:
import ipywidgets as widgets
from IPython.display import display
output = widgets.Output()

In [ ]:
class PauseProcess(object):
    def __init__(self, function):
        import signal
        self.pausebutton = Button(description="Pause")
        display(self.pausebutton)
        self.pausebutton.on_click(self.make_pause)
        self.function = function
    def make_pause(self, event):
        print("Pause")
        signal.pause()
        
    def run_function(self):
        self.function()

pp = PauseProcess(some_function);
pp.run_function()

In [ ]:
some_function()
out

In [ ]:
import asyncio
def wait_for_change(button):
    future = asyncio.Future()
    def getvalue():
        print(Continue)
    button.on_click(getvalue)
    return future




from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

async def f():
    out.append_stdout('did work ' + str(i) + '\n')
    x = await wait_for_change(slider, 'value')
    out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider



In [ ]:
out

In [ ]:
import psutil

psutil.pids()
pid = os.getpid()
print(pid)
p = psutil.Process(pid)
p.name()
p.cmdline()

In [ ]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

In [ ]:
for proc in psutil.process_iter():
    print(proc.name())

In [ ]:
def exitatsigterm(signalNumber, frame):
    raise SystemExit('GUI stop')

In [ ]:
from multiprocessing import Process
import ipywidgets as widgets


        
class MyWidget(widgets.VBox):
    def __init__(self, job):

        self.output = widgets.Output()
        self.run_btn = widgets.Button(description="run")
        self.run_btn.button_style = "info"
        self.run_btn.on_click(self.run)

        self.interupt_button = widgets.Button(description="stop")
        self.interupt_button.on_click(self.stop)
        self.current_process = Process(target=lambda: print("Not set"))
        self.job = job
        super().__init__(
            [
                widgets.HBox([self.run_btn, self.interupt_button]),
                self.output,
            ],
        )

    def run(self, btn):
        if self.current_process.is_alive():
            return

        self.current_process.close()
        with self.output:
            self.run_btn.description = "running ..."
            self.run_btn.button_style = "warning"
            self.current_process = Process(target=self.job, name="main")
            self.current_process.start()
            self.output.clear_output()
            return 0

    def stop(self, btn):
        if not self.current_process.is_alive():
            return

        self.run_btn.description = "stopping"
        self.current_process.terminate()
        self.current_process.join()
        self.current_process.close()
        self.run_btn.description = "stopped"
        self.run_btn.button_style = "danger"



        


MyWidget(some_function)